# Research around Voice Authentication

In [1]:
import os
import numpy as np
import librosa
import pyaudio
import wave
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler

In [2]:
sample_rate = 22050
duration = 5 
num_mfcc = 13
num_recordings = 3
model_path = "voice_auth_model.pkl"

In [3]:
def record_audio(filename, duration=duration, sample_rate=sample_rate):
    """Record audio from the microphone and save to a file."""
    chunk = 1024
    format = pyaudio.paInt16
    channels = 1

    p = pyaudio.PyAudio()
    stream = p.open(format=format, channels=channels, rate=sample_rate, input=True, frames_per_buffer=chunk)

    print(f"Recording: {filename}")
    frames = []

    for _ in range(0, int(sample_rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(format))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    wf.close()

In [4]:
def extract_features(file_path):
    """Extract MFCC features from an audio file."""
    audio, sr = librosa.load(file_path, sr=sample_rate)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=num_mfcc)
    return np.mean(mfccs.T, axis=0)

In [5]:
def train_model(features):
    """Train a GMM model on the extracted features."""
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    model = GaussianMixture(n_components=1, covariance_type='diag', max_iter=200, random_state=42)
    model.fit(features_scaled)
    return model, scaler

In [6]:
def save_model(model, scaler, path=model_path):
    """Save the trained model and scaler to a file."""
    import pickle
    with open(path, 'wb') as f:
        pickle.dump((model, scaler), f)

In [7]:
def load_model(path=model_path):
    """Load the trained model and scaler from a file."""
    import pickle
    with open(path, 'rb') as f:
        return pickle.load(f)

In [8]:
def authenticate_voice(file_path, model, scaler):
    """Authenticate a voice sample against the trained model."""
    features = extract_features(file_path)
    features_scaled = scaler.transform([features])
    score = model.score(features_scaled)
    return score

In [ ]:
def main():
    # Record initial samples for training
    training_files = [f"voice_sample_{i}.wav" for i in range(num_recordings)]
    for file in training_files:
        record_audio(file)

    # Extract features and train model
    features = np.array([extract_features(file) for file in training_files])
    model, scaler = train_model(features)
    save_model(model, scaler)

    # Authenticate a new voice sample
    test_file = "test_voice_sample.wav"
    record_audio(test_file)
    model, scaler = load_model()
    score = authenticate_voice(test_file, model, scaler)

    # Threshold for authentication (this is a simple example, adjust as needed)
    threshold = -50
    if score > threshold:
        print("Authentication successful: Same person")
    else:
        print("Authentication failed: Different person")

if __name__ == "__main__":
    main()

Recording: voice_sample_0.wav
Recording: voice_sample_1.wav
Recording: voice_sample_2.wav


In [3]:
import os
import numpy as np
import pyaudio
import wave
from pydub import AudioSegment
from pydub.playback import play

# Constants
SAMPLE_RATE = 44100
DURATION = 5  # seconds
NUM_RECORDINGS = 3
THRESHOLD = 0.1  # Simple threshold for authentication

def record_audio(filename, duration=DURATION, sample_rate=SAMPLE_RATE):
    """Record audio from the microphone and save to a file."""
    chunk = 1024
    format = pyaudio.paInt16
    channels = 1

    p = pyaudio.PyAudio()
    stream = p.open(format=format, channels=channels, rate=sample_rate, input=True, frames_per_buffer=chunk)

    print(f"Recording: {filename}")
    frames = []

    for _ in range(0, int(sample_rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(format))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    wf.close()

def extract_features(file_path):
    """Extract basic features from an audio file using PyDub."""
    audio = AudioSegment.from_file(file_path)
    samples = np.array(audio.get_array_of_samples())
    return {
        'mean': np.mean(samples),
        'std': np.std(samples),
        'max': np.max(samples),
        'min': np.min(samples)
    }

def authenticate_voice(test_features, reference_features):
    """Authenticate a voice sample against reference features."""
    # Simple Euclidean distance for demonstration
    distance = np.linalg.norm(np.array(list(test_features.values())) - np.array(list(reference_features.values())))
    return distance < THRESHOLD

def main():
    # Record initial samples for training
    training_files = [f"voice_sample_{i}.wav" for i in range(NUM_RECORDINGS)]
    for file in training_files:
        record_audio(file)

    # Extract features from training samples
    reference_features = [extract_features(file) for file in training_files]
    average_reference_features = {key: np.mean([f[key] for f in reference_features]) for key in reference_features[0]}

    # Record a new sample for authentication
    test_file = "test_voice_sample.wav"
    record_audio(test_file)
    test_features = extract_features(test_file)

    # Authenticate the new sample
    if authenticate_voice(test_features, average_reference_features):
        print("Authentication successful: Same person")
    else:
        print("Authentication failed: Different person")

if __name__ == "__main__":
    main()



# Euclidean distance is too basic...

Recording: voice_sample_0.wav
Recording: voice_sample_1.wav
Recording: voice_sample_2.wav
Recording: test_voice_sample.wav
Authentication failed: Different person


In [10]:
# With cosine similarity

import os
import numpy as np
import pyaudio
import wave
from pydub import AudioSegment
from pydub.playback import play
from scipy.spatial.distance import cosine
from scipy import stats
from sklearn.preprocessing import StandardScaler

# Constants
SAMPLE_RATE = 44100
DURATION = 5  # seconds
NUM_RECORDINGS = 3
SIMILARITY_THRESHOLD = 0.85  # Cosine similarity threshold

def record_audio(filename, duration=DURATION, sample_rate=SAMPLE_RATE):
    """Record audio from the microphone and save to a file."""
    chunk = 1024
    format = pyaudio.paInt16
    channels = 1

    p = pyaudio.PyAudio()
    stream = p.open(format=format, channels=channels, rate=sample_rate, input=True, frames_per_buffer=chunk)

    print(f"Recording: {filename}")
    frames = []

    for _ in range(0, int(sample_rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(format))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    wf.close()
def extract_advanced_features(file_path):
    """Extract comprehensive features from an audio file."""
    audio = AudioSegment.from_file(file_path)
    samples = np.array(audio.get_array_of_samples())
    
    # Time-domain features
    features = {
        'mean': np.mean(samples),
        'std': np.std(samples),
        'max': np.max(samples),
        'min': np.min(samples),
        'rms': np.sqrt(np.mean(samples**2)),
        'zero_crossing_rate': np.sum(np.abs(np.diff(np.signbit(samples)))) / len(samples),
        'skewness': stats.skew(samples),
        'kurtosis': stats.kurtosis(samples)
    }
    
    # Frequency-domain features
    fft_features = np.abs(np.fft.fft(samples))
    spectral_centroid = np.average(range(len(fft_features)), weights=fft_features)
    
    features.update({
        'spectral_centroid': spectral_centroid,
        'spectral_bandwidth': np.sqrt(np.average((range(len(fft_features)) - spectral_centroid)**2, weights=fft_features)),
        'spectral_rolloff': np.percentile(fft_features, 85)
    })
    
    return features

class VoiceAuthenticator:
    def __init__(self):
        self.scaler = StandardScaler()
        self.reference_features = None
        
    def train(self, training_files):
        """Train the authenticator with reference voice samples."""
        # Extract features from all training samples
        features_list = [extract_advanced_features(file) for file in training_files]
        
        # Convert features to numpy array
        feature_matrix = np.array([[v for v in f.values()] for f in features_list])
        
        # Fit scaler and transform features
        self.scaler.fit(feature_matrix)
        normalized_features = self.scaler.transform(feature_matrix)
        
        # Store average normalized features as reference
        self.reference_features = np.mean(normalized_features, axis=0)
        
    def authenticate(self, test_file):
        """Authenticate a voice sample against stored reference features."""
        # Extract and normalize test features
        test_features = extract_advanced_features(test_file)
        test_vector = np.array([[v for v in test_features.values()]])
        normalized_test = self.scaler.transform(test_vector)
        
        # Calculate cosine similarity
        similarity = 1 - cosine(self.reference_features, normalized_test[0])
        
        # Calculate Euclidean distance for additional verification
        euclidean_dist = np.linalg.norm(self.reference_features - normalized_test[0])
        
        return {
            'authenticated': similarity > SIMILARITY_THRESHOLD,
            'confidence': similarity,
            'euclidean_distance': euclidean_dist
        }

def main():
    # Record initial samples for training (if not already recorded)
    training_files = [f"voice_sample_{i}.wav" for i in range(NUM_RECORDINGS)]
    for file in training_files:
        if not os.path.exists(file):  # Only record if the file doesn't exist
            print(f"Recording training sample: {file}")
            record_audio(file)
        else:
            print(f"Training sample already exists: {file}")
    
    # Initialize and train the authenticator
    authenticator = VoiceAuthenticator()
    authenticator.train(training_files)
    
    # Record a test sample (if not already recorded)
    test_file = "test_voice_sample.wav"
    if not os.path.exists(test_file):  # Only record if the file doesn't exist
        print(f"Recording test sample: {test_file}")
        record_audio(test_file)
    else:
        print(f"Test sample already exists: {test_file}")
    
    # Authenticate the test sample
    result = authenticator.authenticate(test_file)
    
    # Display results
    print("\nAuthentication Results:")
    print(f"Status: {'Successful' if result['authenticated'] else 'Failed'}")
    print(f"Confidence Score: {result['confidence']:.2f}")
    print(f"Euclidean Distance: {result['euclidean_distance']:.2f}")

if __name__ == "__main__":
    main()

Training sample already exists: voice_sample_0.wav
Training sample already exists: voice_sample_1.wav
Training sample already exists: voice_sample_2.wav
Test sample already exists: test_voice_sample.wav

Authentication Results:
Status: Failed
Confidence Score: -0.21
Euclidean Distance: 7.86


In [2]:
import os
import numpy as np
import pyaudio
import wave
from pydub import AudioSegment
from pydub.playback import play
from scipy.spatial.distance import cosine
from scipy import stats
from sklearn.preprocessing import StandardScaler

# Constants
SAMPLE_RATE = 44100  # Sample rate for audio recording
SIMILARITY_THRESHOLD = 0.85  # Cosine similarity threshold

def record_audio(filename, duration, sample_rate=SAMPLE_RATE):
    """Record audio from the microphone and save to a file."""
    chunk = 1024
    format = pyaudio.paInt16
    channels = 1

    p = pyaudio.PyAudio()
    stream = p.open(format=format, channels=channels, rate=sample_rate, input=True, frames_per_buffer=chunk)

    print(f"Recording: {filename}")
    frames = []

    for _ in range(0, int(sample_rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(format))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    wf.close()

def extract_advanced_features(file_path):
    """Extract comprehensive features from an audio file."""
    audio = AudioSegment.from_file(file_path)
    samples = np.array(audio.get_array_of_samples())
    
    # Time-domain features
    features = {
        'mean': np.mean(samples),
        'std': np.std(samples),
        'max': np.max(samples),
        'min': np.min(samples),
        'rms': np.sqrt(np.mean(samples**2)),
        'zero_crossing_rate': np.sum(np.abs(np.diff(np.signbit(samples)))) / len(samples),
        'skewness': stats.skew(samples),
        'kurtosis': stats.kurtosis(samples)
    }
    
    # Frequency-domain features
    fft_features = np.abs(np.fft.fft(samples))
    spectral_centroid = np.average(range(len(fft_features)), weights=fft_features)
    
    features.update({
        'spectral_centroid': spectral_centroid,
        'spectral_bandwidth': np.sqrt(np.average((range(len(fft_features)) - spectral_centroid)**2, weights=fft_features)),
        'spectral_rolloff': np.percentile(fft_features, 85)
    })
    
    return features

class VoiceAuthenticator:
    def __init__(self):
        self.scaler = StandardScaler()
        self.reference_features = None
        
    def train(self, training_files):
        """Train the authenticator with reference voice samples."""
        # Extract features from all training samples
        features_list = [extract_advanced_features(file) for file in training_files]
        
        # Convert features to numpy array
        feature_matrix = np.array([[v for v in f.values()] for f in features_list])
        
        # Fit scaler and transform features
        self.scaler.fit(feature_matrix)
        normalized_features = self.scaler.transform(feature_matrix)
        
        # Store average normalized features as reference
        self.reference_features = np.mean(normalized_features, axis=0)
        
    def authenticate(self, test_file):
        """Authenticate a voice sample against stored reference features."""
        # Extract and normalize test features
        test_features = extract_advanced_features(test_file)
        test_vector = np.array([[v for v in test_features.values()]])
        normalized_test = self.scaler.transform(test_vector)
        
        # Calculate cosine similarity
        similarity = 1 - cosine(self.reference_features, normalized_test[0])
        
        # Calculate Euclidean distance for additional verification
        euclidean_dist = np.linalg.norm(self.reference_features - normalized_test[0])
        
        return {
            'authenticated': similarity > SIMILARITY_THRESHOLD,
            'confidence': similarity,
            'euclidean_distance': euclidean_dist
        }

def main():
    # Take user inputs
    num_recordings = int(input("Enter the number of training recordings: "))
    duration = int(input("Enter the duration of each recording (in seconds): "))
    
    # Record training samples
    training_files = []
    for i in range(num_recordings):
        filename = input(f"Enter filename for training recording {i + 1} (e.g., voice_sample_{i}.wav): ")
        if not os.path.exists(filename):  # Only record if the file doesn't exist
            print(f"Recording training sample: {filename}")
            record_audio(filename, duration)
        else:
            print(f"Training sample already exists: {filename}")
        training_files.append(filename)
    
    # Initialize and train the authenticator
    authenticator = VoiceAuthenticator()
    authenticator.train(training_files)
    
    # Record test sample
    test_file = input("Enter filename for the test recording (e.g., test_voice_sample.wav): ")
    if not os.path.exists(test_file):  # Only record if the file doesn't exist
        print(f"Recording test sample: {test_file}")
        record_audio(test_file, duration)
    else:
        print(f"Test sample already exists: {test_file}")
    
    # Authenticate the test sample
    result = authenticator.authenticate(test_file)
    
    # Display results
    print("\nAuthentication Results:")
    print(f"Status: {'Successful' if result['authenticated'] else 'Failed'}")
    print(f"Confidence Score: {result['confidence']:.2f}")
    print(f"Euclidean Distance: {result['euclidean_distance']:.2f}")

if __name__ == "__main__":
    main()

Recording training sample: hello.wav
Recording: hello.wav
Recording training sample: canyouhearme.wav
Recording: canyouhearme.wav
Recording training sample: takeoff.wav
Recording: takeoff.wav
Recording test sample: test1.wav
Recording: test1.wav

Authentication Results:
Status: Failed
Confidence Score: -0.04
Euclidean Distance: 8.88


In [9]:
import os
import numpy as np
import pyaudio
import wave
from pydub import AudioSegment
from scipy.spatial.distance import cosine
from scipy.signal import find_peaks
from sklearn.preprocessing import StandardScaler
from scipy.fftpack import fft

# Constants
SAMPLE_RATE = 16000  # Lower sample rate for faster processing
SIMILARITY_THRESHOLD = 0.85  # Cosine similarity threshold
CHUNK = 1024  # Audio buffer size
FORMAT = pyaudio.paInt16  # Audio format
CHANNELS = 1  # Mono audio

def record_audio(filename, duration, sample_rate=SAMPLE_RATE):
    """Record audio from the microphone and save to a file."""
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=sample_rate, input=True, frames_per_buffer=CHUNK)

    print(f"Recording: {filename}")
    frames = [stream.read(CHUNK) for _ in range(0, int(sample_rate / CHUNK * duration))]

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    wf.close()

def extract_features(file_path):
    """Extract essential features from an audio file."""
    audio = AudioSegment.from_file(file_path, format="wav")
    samples = np.array(audio.get_array_of_samples(), dtype=np.float32)

    # Normalize samples
    samples /= np.max(np.abs(samples))

    # Time-domain features
    features = {
        'mean': np.mean(samples),
        'std': np.std(samples),
        'rms': np.sqrt(np.mean(samples ** 2)),
        'zcr': np.mean(np.abs(np.diff(np.sign(samples)))),
    }

    # Frequency-domain features using FFT
    fft_spectrum = np.abs(fft(samples))[:len(samples) // 2]
    fft_freqs = np.fft.fftfreq(len(samples), 1 / SAMPLE_RATE)[:len(samples) // 2]
    peaks, _ = find_peaks(fft_spectrum, height=np.max(fft_spectrum) * 0.1)

    if np.sum(fft_spectrum) > 0:
        spectral_centroid = np.sum(fft_freqs * fft_spectrum) / np.sum(fft_spectrum)
        spectral_bandwidth = np.sqrt(np.sum((fft_freqs - spectral_centroid) ** 2 * fft_spectrum) / np.sum(fft_spectrum))
    else:
        spectral_centroid = 0
        spectral_bandwidth = 0

    features.update({
        'spectral_centroid': spectral_centroid,
        'spectral_bandwidth': spectral_bandwidth,
        'peak_count': len(peaks),
    })

    return features

class VoiceAuthenticator:
    def __init__(self):
        self.scaler = StandardScaler()
        self.reference_features = None

    def train(self, training_files):
        """Train the authenticator with reference voice samples."""
        features_list = [extract_features(file) for file in training_files]

        # Convert features to a numpy array
        feature_matrix = np.array([[v for v in f.values()] for f in features_list])

        # Normalize features
        self.scaler.fit(feature_matrix)
        normalized_features = self.scaler.transform(feature_matrix)

        # Store average features
        self.reference_features = np.mean(normalized_features, axis=0)

    def authenticate(self, test_file):
        """Authenticate a voice sample against stored reference features."""
        test_features = extract_features(test_file)
        test_vector = np.array([[v for v in test_features.values()]])
        normalized_test = self.scaler.transform(test_vector)

        # Calculate similarity metrics
        similarity = 1 - cosine(self.reference_features, normalized_test[0])
        euclidean_dist = np.linalg.norm(self.reference_features - normalized_test[0])

        return {
            'authenticated': similarity > SIMILARITY_THRESHOLD,
            'confidence': similarity,
            'euclidean_distance': euclidean_dist
        }

def main():
    num_recordings = int(input("Enter the number of training recordings: "))
    duration = int(input("Enter the duration of each recording (in seconds): "))

    # Record training samples
    training_files = []
    for i in range(num_recordings):
        filename = f"voice_sample_{i + 1}.wav"
        if not os.path.exists(filename):
            record_audio(filename, duration)
        training_files.append(filename)

    # Initialize and train the authenticator
    authenticator = VoiceAuthenticator()
    authenticator.train(training_files)

    # Record test sample
    test_file = "test_voice_sample.wav"
    if not os.path.exists(test_file):
        record_audio(test_file, duration)

    # Authenticate the test sample
    result = authenticator.authenticate(test_file)

    # Display results
    print("\nAuthentication Results:")
    print(f"Status: {'Successful' if result['authenticated'] else 'Failed'}")
    print(f"Confidence Score: {result['confidence']:.2f}")
    print(f"Euclidean Distance: {result['euclidean_distance']:.2f}")

if __name__ == "__main__":
    main()


Recording: voice_sample_1.wav
Recording: voice_sample_2.wav
Recording: voice_sample_3.wav
Recording: test_voice_sample.wav

Authentication Results:
Status: Failed
Confidence Score: 0.14
Euclidean Distance: 5.33


In [11]:
import os
import numpy as np
import pyaudio
import wave
from pydub import AudioSegment
from scipy.spatial.distance import cosine
from scipy.signal import find_peaks
from sklearn.preprocessing import StandardScaler
from scipy.fftpack import fft

# Constants
SAMPLE_RATE = 16000
SIMILARITY_THRESHOLD = 0.85
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1


def record_audio(filename, duration, sample_rate=SAMPLE_RATE):
    """Record audio from the microphone and save to a file."""
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=sample_rate, input=True, frames_per_buffer=CHUNK)

    print(f"Recording: {filename}")
    frames = [stream.read(CHUNK) for _ in range(0, int(sample_rate / CHUNK * duration))]

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    wf.close()


def extract_features(file_path):
    """Extract essential features from an audio file."""
    audio = AudioSegment.from_file(file_path, format="wav")
    samples = np.array(audio.get_array_of_samples(), dtype=np.float32)

    # Normalize samples
    samples /= np.max(np.abs(samples))

    # Time-domain features
    features = {
        'mean': np.mean(samples),
        'std': np.std(samples),
        'rms': np.sqrt(np.mean(samples ** 2)),
        'zcr': np.mean(np.abs(np.diff(np.sign(samples)))),
    }

    # Frequency-domain features using FFT
    fft_spectrum = np.abs(fft(samples))[:len(samples) // 2]
    fft_freqs = np.fft.fftfreq(len(samples), 1 / SAMPLE_RATE)[:len(samples) // 2]
    peaks, _ = find_peaks(fft_spectrum, height=np.max(fft_spectrum) * 0.1)

    if np.sum(fft_spectrum) > 0:
        spectral_centroid = np.sum(fft_freqs * fft_spectrum) / np.sum(fft_spectrum)
        spectral_bandwidth = np.sqrt(np.sum((fft_freqs - spectral_centroid) ** 2 * fft_spectrum) / np.sum(fft_spectrum))
    else:
        spectral_centroid = 0
        spectral_bandwidth = 0

    features.update({
        'spectral_centroid': spectral_centroid,
        'spectral_bandwidth': spectral_bandwidth,
        'peak_count': len(peaks),
    })

    return features


class VoiceAuthenticator:
    def __init__(self):
        self.scaler = StandardScaler()
        self.reference_features = None

    def train(self, training_files):
        """Train the authenticator with reference voice samples."""
        features_list = [extract_features(file) for file in training_files]

        # Convert features to a numpy array
        feature_matrix = np.array([[v for v in f.values()] for f in features_list])

        # Normalize features
        self.scaler.fit(feature_matrix)
        normalized_features = self.scaler.transform(feature_matrix)

        # Store average features
        self.reference_features = np.mean(normalized_features, axis=0)

    def authenticate(self, test_file):
        """Authenticate a voice sample against stored reference features."""
        test_features = extract_features(test_file)
        test_vector = np.array([[v for v in test_features.values()]])
        normalized_test = self.scaler.transform(test_vector)

        # Calculate similarity metrics
        similarity = 1 - cosine(self.reference_features, normalized_test[0])
        euclidean_dist = np.linalg.norm(self.reference_features - normalized_test[0])

        return {
            'authenticated': similarity > SIMILARITY_THRESHOLD,
            'confidence': similarity,
            'euclidean_distance': euclidean_dist
        }


def main():
    duration = int(input("Enter the duration of each recording (in seconds): "))

    # Record training samples
    training_files = []
    print("Recording training samples...")
    for i in range(3):  # Collect exactly 3 samples for simplicity
        filename = f"voice_sample_{i + 1}.wav"
        if not os.path.exists(filename):
            record_audio(filename, duration)
        training_files.append(filename)

    # Initialize and train the authenticator
    authenticator = VoiceAuthenticator()
    authenticator.train(training_files)

    # Record test sample
    print("\nRecording test sample...")
    test_file = "test_voice_sample.wav"
    record_audio(test_file, duration)

    # Authenticate the test sample
    result = authenticator.authenticate(test_file)

    # Display results
    print("\nAuthentication Results:")
    print(f"Status: {'Successful' if result['authenticated'] else 'Failed'}")
    print(f"Confidence Score: {result['confidence']:.2f}")
    print(f"Euclidean Distance: {result['euclidean_distance']:.2f}")


if __name__ == "__main__":
    main()

Recording training samples...
Recording: voice_sample_1.wav
Recording: voice_sample_2.wav
Recording: voice_sample_3.wav

Recording test sample...
Recording: test_voice_sample.wav

Authentication Results:
Status: Failed
Confidence Score: 0.23
Euclidean Distance: 20.71
